# CS418 Project1 - Exploratory Data Analysis

Find the Project Description [here](./Project_Description.pdf).

This project is done as part of **CS418 - Introduction to DataScience** at UIC.


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.stats as st

### Load Dataset

In [2]:
elect = pd.read_csv('data/election_train.csv')
demographics_train = pd.read_csv('data/demographics_train.csv')

In [3]:
print(elect.shape)
elect.head()

(2405, 6)


,Year,State,County,Office,Party,Votes
0,2018,AZ,Apache County,US Senator,Democratic,16298
1,2018,AZ,Apache County,US Senator,Republican,7810
2,2018,AZ,Cochise County,US Senator,Democratic,17383
3,2018,AZ,Cochise County,US Senator,Republican,26929
4,2018,AZ,Coconino County,US Senator,Democratic,34240


In [4]:
print(demographics_train.shape)
demographics_train.head()

(1216, 17)


,State,County,FIPS,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
0,Wisconsin,La Crosse,55063,117538,0,90.537528,1.214075,1.724549,2.976059,51.171536,43.241335,14.702479,51477,4.796952,5.474767,67.529757,16.827753
1,Virginia,Alleghany,51005,15919,12705,91.940449,5.207614,1.432251,1.300333,51.077329,31.660280,23.902255,45538,4.560986,15.537543,83.711604,52.393846
2,Indiana,Fountain,18045,16741,12750,95.705155,0.400215,2.359477,1.547100,49.770026,35.899887,18.941521,45924,7.978789,12.032155,85.538940,65.951276
3,Ohio,Geauga,39055,94020,0,95.837056,1.256116,1.294405,2.578175,50.678579,36.281642,18.028079,74165,4.036902,8.928599,62.730824,63.968990
4,Wisconsin,Jackson,55053,20566,15835,86.662453,1.983857,3.082758,1.376058,46.649810,36.292911,17.587280,49608,5.569698,11.792912,86.129256,72.238251


**1. (5 pts.) Reshape dataset election_train from long format to wide format. Hint: the reshaped dataset should contain 1205 rows and 6 columns.**

In [5]:
election_train = elect.pivot(index=['Year','State','County','Office'], columns='Party', values='Votes').reset_index()
print(election_train.shape)
election_train

(1205, 6)


Party,Year,State,County,Office,Democratic,Republican
0,2018,AZ,Apache County,US Senator,16298.0,7810.0
1,2018,AZ,Cochise County,US Senator,17383.0,26929.0
2,2018,AZ,Coconino County,US Senator,34240.0,19249.0
3,2018,AZ,Gila County,US Senator,7643.0,12180.0
4,2018,AZ,Graham County,US Senator,3368.0,6870.0
...,...,...,...,...,...,...
1200,2018,WY,Platte County,US Senator,801.0,2850.0
1201,2018,WY,Sublette County,US Senator,668.0,2653.0
1202,2018,WY,Sweetwater County,US Senator,3943.0,8577.0
1203,2018,WY,Uinta County,US Senator,1371.0,4713.0


**2. Merge reshaped dataset election_train with dataset demographics_train. Make sure that you address all inconsistencies in the names of the states and the counties before merging. Hint: the merged dataset should contain 1200 rows.**

In [6]:
state_abbr = {'AL': 'Alabama',
 'AK': 'Alaska',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'MP': 'Northern Mariana Islands',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PW': 'Palau',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'PR': 'Puerto Rico',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VA': 'Virginia',
 'WA': 'Washington',
 'DC': 'Washington, DC',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming',
 'VI': 'Virgin Islands'}

In [7]:
election_train['State'] = election_train['State'].map(state_abbr)

In [8]:
def standardize_county_name(county):
    county = county.replace('County', '').strip()
    return county.lower()
    
election_train['County'] = election_train['County'].apply(standardize_county_name)
demographics_train['County'] = demographics_train['County'].apply(standardize_county_name)

In [9]:
election_train_merge = election_train.merge(demographics_train, how='inner', on=['State', 'County'])
election_train_merge.shape

(1200, 21)

**3. (5 pts.) Explore the merged dataset. How many variables does the dataset have? What is the type of these variables? Are there any irrelevant or redundant variables? If so, how will you deal with these variables?**

In [10]:
print('Shape: ', election_train_merge.shape)
election_train_merge.info()

Shape:  (1200, 21)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 0 to 1199
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Year                                   1200 non-null   int64  
 1   State                                  1200 non-null   object 
 2   County                                 1200 non-null   object 
 3   Office                                 1200 non-null   object 
 4   Democratic                             1197 non-null   float64
 5   Republican                             1198 non-null   float64
 6   FIPS                                   1200 non-null   int64  
 7   Total Population                       1200 non-null   int64  
 8   Citizen Voting-Age Population          1200 non-null   int64  
 9   Percent White, not Hispanic or Latino  1200 non-null   float64
 10  Percent Black, not Hispanic or Latino  1200 non-null 

In [11]:
election_train_merge.describe()

,Year,Democratic,Republican,FIPS,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
count,1200.0,1197.000000,1198.000000,1200.000000,1.200000e+03,1.200000e+03,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,2018.0,25096.309106,20436.841402,38315.355000,1.208766e+05,3.226592e+04,79.099685,5.552660,10.556249,5.072053,49.825231,36.753853,18.105368,50098.516667,6.540929,13.448139,78.629091,55.950082
std,0.0,72593.640184,45218.050721,13001.996705,3.183773e+05,1.247969e+05,19.782542,9.274481,15.840002,6.054644,2.394356,5.618738,4.758807,12291.769508,2.776238,6.439020,9.191006,32.145514
min,2018.0,6.000000,46.000000,4001.000000,7.600000e+01,0.000000e+00,2.776702,0.000000,0.000000,0.000000,21.513413,11.842105,6.653188,21190.000000,0.000000,2.134454,26.335440,0.000000
25%,2018.0,1427.000000,2667.500000,27146.500000,1.208350e+04,0.000000e+00,70.168347,0.538615,1.819812,1.468692,49.352173,33.298306,15.052983,42081.500000,4.710440,8.916211,74.432551,29.744143
50%,2018.0,4213.000000,6691.000000,39140.000000,3.264300e+04,0.000000e+00,86.801005,1.605082,3.884901,2.869445,50.310516,36.427490,17.704921,48391.000000,6.417206,11.841157,81.013166,56.201291
75%,2018.0,14206.000000,16740.500000,48416.000000,8.582300e+04,1.893250e+04,93.876656,6.426589,11.075522,6.319101,50.971969,39.451774,20.501641,54958.250000,8.118306,16.811195,85.004467,83.377538
max,2018.0,881802.000000,672505.000000,56043.000000,4.434257e+06,2.723565e+06,99.627329,63.953279,95.479801,52.229868,56.418468,67.367823,37.622759,125672.000000,18.771186,49.673777,97.014925,100.000000


**Merged Dataset Summary:**
- **Number of variables**: 21
- **Types of data:** float object: 13, int object: 5, string object: 3
- **Irrelevant or redundant variables?**
    Office and Year are the irrelevant or redundant variables

- **Dealing with Irrelevant/Redundant Variables**:
    These variables can be removed from the dataframe since they are of no signifant importance for further analysis


In [12]:
election_train_merge=election_train_merge.drop(columns=['Office','Year'])

**4. (10 pts.) Search the merged dataset for missing values. Are there any missing values? If so, how will you deal with these values?**

In [13]:
election_train_merge[pd.isna(election_train_merge['Democratic'])]

,State,County,Democratic,Republican,FIPS,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
425,Nebraska,lancaster,NaN,49449.0,31109,301707,0,82.659667,3.783472,6.531834,7.523856,49.891782,45.457016,12.175057,53730,4.372984,6.636272,62.697684,8.253126
714,Tennessee,meigs,NaN,2694.0,47121,11804,0,94.713656,1.330058,1.465605,1.067435,50.626906,33.361572,19.662826,35209,12.544170,21.141176,91.176471,100.000000
865,Texas,menard,NaN,632.0,48327,2163,0,56.310680,1.248266,39.389736,4.299584,46.833102,36.430883,22.468793,37917,8.360836,21.113990,84.909326,100.000000


In [14]:
election_train_merge[pd.isna(election_train_merge['Republican'])]

,State,County,Democratic,Republican,FIPS,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
750,Texas,bee,2811.0,NaN,48025,32706,0,32.660674,7.989360,57.909864,4.344769,39.579894,40.448236,11.514707,44005,6.065048,28.602944,91.094503,42.977308
1114,Wisconsin,lafayette,3592.0,NaN,55065,16793,0,94.771631,0.339427,3.507414,2.286667,49.199071,37.688323,16.709343,53038,2.998885,10.123457,82.336861,100.000000


In [15]:
election_train_merge[election_train_merge['Citizen Voting-Age Population']==0]

,State,County,Democratic,Republican,FIPS,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
0,Arizona,apache,16298.0,7810.0,4001,72346,0,18.571863,0.486551,5.947806,1.719515,50.598513,45.854643,13.322091,32460,15.807433,21.758252,88.941063,74.061076
3,Arizona,gila,7643.0,12180.0,4007,53179,0,63.222325,0.552850,18.548675,4.249798,50.296170,32.238290,26.397638,40593,12.129932,15.729958,82.262624,41.062000
4,Arizona,graham,3368.0,6870.0,4009,37529,0,51.461536,1.811932,32.097844,4.385942,46.313518,46.393456,12.315809,47422,14.424104,14.580797,86.675944,46.437399
7,Arizona,mohave,19214.0,50209.0,4015,203629,0,78.252606,0.951731,15.708470,6.969047,49.676618,30.485835,26.858650,39856,11.680953,16.145850,88.121178,22.963644
9,Arizona,pima,221242.0,160550.0,4019,1003338,0,53.271579,3.199719,36.105978,12.903428,50.807405,40.087388,17.801778,46764,9.214114,12.252238,69.199391,7.523491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,Wyoming,converse,834.0,3959.0,56009,14223,0,88.849047,0.007031,7.691767,2.706883,49.933207,38.515081,13.668003,66737,5.282284,9.758393,84.468152,55.360370
1190,Wyoming,goshen,1020.0,3658.0,56015,13546,0,86.409272,0.147645,10.519711,2.724051,47.091392,35.914661,20.389783,44883,6.918819,8.390574,77.036880,45.995924
1192,Wyoming,lincoln,1152.0,5846.0,56023,18543,0,92.600982,0.210322,4.416761,2.151755,48.773122,38.715418,14.382786,64579,5.618095,6.949996,79.631291,82.729482
1196,Wyoming,sublette,668.0,2653.0,56035,10032,0,91.646730,0.000000,7.814992,2.053429,46.949761,36.393541,13.337321,76004,2.786971,4.658830,75.645069,100.000000


**Missing values information** :
There 3 rows with missing values for 'Democratic' votes and 2 rows with missing values for 'Republican' votes.       Additionally, 680 rows are present with 'Citizen Voting-Age Population' as 0. These can be considered as a missing value.

**Dealing with Missing values**: The total 5 rows with missing votes for one party should be deleted as it can influence the result but both the party votes are unknown. Since there are 680 rows missing in the 'Citizen Voting-Age Population', it can be best resolved by deleting the feature from the data-frame

In [16]:
election_train_merge=election_train_merge.dropna().drop(columns=['Citizen Voting-Age Population'])
election_train_merge['Democratic']=election_train_merge['Democratic'].astype(int)
election_train_merge['Republican']=election_train_merge['Republican'].astype(int)
election_train_merge

,State,County,Democratic,Republican,FIPS,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
0,Arizona,apache,16298,7810,4001,72346,18.571863,0.486551,5.947806,1.719515,50.598513,45.854643,13.322091,32460,15.807433,21.758252,88.941063,74.061076
1,Arizona,cochise,17383,26929,4003,128177,56.299492,3.714395,34.403208,11.458374,49.069646,37.902276,19.756275,45383,8.567108,13.409171,76.837055,36.301067
2,Arizona,coconino,34240,19249,4005,138064,54.619597,1.342855,13.711033,4.825298,50.581614,48.946141,10.873943,51106,8.238305,11.085381,65.791439,31.466066
3,Arizona,gila,7643,12180,4007,53179,63.222325,0.552850,18.548675,4.249798,50.296170,32.238290,26.397638,40593,12.129932,15.729958,82.262624,41.062000
4,Arizona,graham,3368,6870,4009,37529,51.461536,1.811932,32.097844,4.385942,46.313518,46.393456,12.315809,47422,14.424104,14.580797,86.675944,46.437399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,Wyoming,platte,801,2850,56031,8740,89.359268,0.057208,7.814645,2.780320,47.711670,32.700229,22.013730,41051,3.901047,9.675889,80.300395,58.647744
1196,Wyoming,sublette,668,2653,56035,10032,91.646730,0.000000,7.814992,2.053429,46.949761,36.393541,13.337321,76004,2.786971,4.658830,75.645069,100.000000
1197,Wyoming,sweetwater,3943,8577,56037,44812,79.815674,0.865840,15.859591,5.509685,47.824244,44.153352,9.417120,68233,5.072255,9.314606,78.628507,10.916313
1198,Wyoming,uinta,1371,4713,56041,20893,87.718375,0.186665,8.959939,3.986981,49.327526,43.205858,10.678218,53323,6.390755,10.361224,81.793082,43.095937


**5. (5 pts.) Create a new variable named “Party” that labels each county as Democratic or
Republican. This new variable should be equal to 1 if there were more votes cast for the
Democratic party than the Republican party in that county and it should be equal to 0
otherwise**

In [17]:
election_train_merge['Party'] = (election_train_merge['Democratic'] > election_train_merge['Republican']).astype(int)
election_train_merge.sample(5)

,State,County,Democratic,Republican,FIPS,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural,Party
140,Indiana,warren,914,2208,18171,8309,96.726441,0.686003,1.359971,0.421230,50.427248,33.590083,19.304369,58623,5.002349,9.030949,80.432944,77.103902,0
607,Ohio,tuscarawas,14088,18244,39157,92579,95.155489,0.804718,2.349345,1.650482,50.823621,36.475875,17.836658,46992,5.734798,14.068037,85.020218,41.580437,0
570,Ohio,hancock,10929,17123,39063,75672,89.939476,1.354530,5.067925,3.252194,50.893329,39.036896,15.830162,51604,5.476220,7.505079,74.161978,31.110428,0
1188,Wyoming,converse,834,3959,56009,14223,88.849047,0.007031,7.691767,2.706883,49.933207,38.515081,13.668003,66737,5.282284,9.758393,84.468152,55.360370,0
886,Texas,red river,973,3427,48387,12455,73.263750,17.406664,7.193898,2.312324,52.492975,33.063027,22.729827,35016,6.915816,19.557690,86.585502,75.730949,0


**6. (10 pts.) Compute the mean median household income for Democratic counties and
Republican counties. Which one is higher? Perform a hypothesis test to determine
whether this difference is statistically significant at the 𝜶 = 𝟎. 𝟎𝟓 significance level. What
is the result of the test? What conclusion do you make from this result?**

In [18]:
mean_income_democratic = election_train_merge[election_train_merge['Party'] == 1]['Median Household Income'].mean()
mean_income_republican = election_train_merge[election_train_merge['Party'] == 0]['Median Household Income'].mean()
print("Mean 'Median Household income' of Democratic County's:", mean_income_democratic)
print("Mean 'Median Household income' of Republican County's:", mean_income_republican)

Mean 'Median Household income' of Democratic County's: 53798.732307692306
Mean 'Median Household income' of Republican County's: 48746.81954022989


**Mean 'Median Household Income' of Democratic Counties are higher than Republican Counties.**

**Null Hypotheses:** Median Household Income of Democratic counties is equal to than Republican counties.(*μd==μr*)<br />
**Alternative Hypotheses:** Median Household Income of Democratic counties are higher than Republican counties. (*μd>μr*) <br/><br/>
We do a t-test on the data since population standard deviation is unknown.<br/>
We do a right tailed t-test since the alternative hypothesis is μd>μr

In [21]:
(t_test_statistic,p_value)=st.ttest_ind(election_train_merge[election_train_merge['Party'] == 1]['Median Household Income'],election_train_merge[election_train_merge['Party'] == 0]['Median Household Income'], equal_var=False)

In [22]:
#Since the function return two sided test result, convert it into right tailed test.
p_value=p_value/2

In [23]:
t_test_statistic

5.479141589767387

In [24]:
p_value

3.574718681591299e-08

*The p value for the Null hypothesis is 3.57*10^-8 which is way lesser than the significance level 0.05.<br/>
  Hence, we reject the null hypothesis and there is sufficient evidence to conclude that Median Household Income of Democratic counties may be higher than that of the republican ones*

 **7. (10 pts.) Compute the mean population for Democratic counties and Republican counties. Which one is higher? Perform a hypothesis test to determine whether this difference is statistically significant at the 𝜶=𝟎.𝟎𝟓 significance level. What is the result of the test? What conclusion do you make from this result?**